<a href="https://colab.research.google.com/github/labnac/scuolaAIP/blob/main/4_autoencoder_nonlinear_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import dei moduli

In [ ]:
import warnings
from keras.models import Sequential
from keras.layers import Dense
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers, optimizers, callbacks
from keras.callbacks import TensorBoard
import tensorflow as tf
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_squared_error

INIZIALIZZAZIONE DEI PARAMETRI: Parametri del test

In [ ]:
rho = 0.8 #attendibilità del test

INIZIALIZZAZIONE DEI PARAMETRI: Parametri del fattore

In [ ]:
kappa = 0 #media fattoriale
phi = 1 #varianza fattoriale

INIZIALIZZAZIONE DEI PARAMETRI: Parametri della relazione tra item e fattore



In [ ]:
t = 0  #intercetta
l = 0.8 #loading

INIZIALIZZAZIONE DEI PARAMETRI: Parametri del dataset

In [ ]:
sample_size = 800 #numerosità campionaria
n_lin = 4 #numero di item lineari
n_sig = 4 #numero di item non lineari
a = 3 #pendenza della sigmoide
b = -1 #location della sigmoide

Funzione per la creazione del dataset

In [ ]:
def dataset(sample_size, rho, kappa, phi, t, l, a, b, n_lin, n_sig):
    np.random.seed(seed=2022)
    ### Simulazione dei punteggi fattoriali e dei dataset ###
    NL_X = []
    KSI = []
    X = []
    n_var = n_lin+n_sig
    epsvar = ((1-rho)*phi)/rho #varianza dei residui

    for i in range(sample_size):
      ksi = kappa + math.sqrt(phi)*np.random.normal(0,1)
      for s in range(n_sig):
        nl_x = (((1/(1+np.exp(-a*(ksi-(b)))))*8) - 4) + (math.sqrt(epsvar)*np.random.normal(0, 1))#0.3*(((1/(1+np.exp(-a*(ksi-(b))))*2)-1)*10 +  0.8*ksi + (math.sqrt(epsvar)*np.random.normal(0, 1)))
        NL_X.append(nl_x)
        KSI.append(ksi)
      for k in range(n_lin):
          x = t + 0.8*ksi  + (math.sqrt(epsvar)*np.random.normal(0, 1))
          X.append(x)
          KSI.append(ksi)

    #punteggi fattoriali#
    KSI = np.array(KSI)
    ksi_array = KSI.reshape(sample_size,n_var)

    #dataset
    NL_X = np.array(NL_X)
    NL_X = NL_X.reshape(sample_size, n_sig)
    X = np.array(X)
    X = X.reshape(sample_size, n_lin)
    dataset_array = np.append(NL_X, X, axis=1)
    dataset_df = pd.DataFrame(dataset_array)

    return ksi_array, dataset_df

KSI, Dataset = dataset(sample_size, rho, kappa, phi, t, l, a, b, n_lin, n_sig)

np.save('punteggi_fattoriali.npy',KSI)
np.save('Dataset',Dataset)

In [ ]:
KSI = np.load('punteggi_fattoriali.npy')
Dataset = np.load('Dataset.npy')

Funzione per la creazione del modello di autoencoder

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)


def create_autoencoder(input_dim, l_rate, name='autoencoder'):
    # Create model
    autoencoder = Sequential(name=name)

    # Encoder
    autoencoder.add(Dense(5, input_dim=input_dim, activation='sigmoid'))

    # Internal layer
    autoencoder.add(Dense(1, activation='linear'))

    # Decoder
    autoencoder.add(Dense(5, activation='sigmoid'))

    # Output layer
    autoencoder.add(Dense(input_dim, activation='linear'))

    # Compile model
    autoencoder.compile(loss='mean_squared_error',
                        optimizer=tf.keras.optimizers.Adam(learning_rate = l_rate))

    return autoencoder

In [ ]:
#Generazione dei punteggi fattoriali e del dataset

#KSI, Dataset = dataset(sample_size, rho, kappa, phi, t, l, a, b, n_lin, n_sig)

#Creazione di un autoencoder basato sul modello precedente che analizzi il nostro dataset

n_features = Dataset.shape[1] #definizione della dimensionalità dell'input sulla base del numero di colonne del dataset
l_rate = 5e-3 #tasso di apprendimento

autoencoder_model = create_autoencoder(n_features, l_rate)

autoencoder_model.summary()

history_dict = {}

# TensorBoard Callback
cb = TensorBoard()

history_callback = autoencoder_model.fit(Dataset, Dataset,
                              batch_size=4,
                              epochs=100,
                              verbose=1,
                              callbacks=[cb])

score = autoencoder_model.evaluate(Dataset, Dataset, verbose=0) #variabile che restituisce la media delle loss

history_dict[autoencoder_model.name] = [history_callback, autoencoder_model]

print(history_callback)
print(score)


In [ ]:
loss = history_dict[autoencoder_model.name][0].history['loss']
plt.plot(loss, label="autoencoder_model"+ " loss")
plt.ylabel('loss')
plt.xlabel('epochs')

Funzioni per ottenere l'output dello strato interno e l'output finale dell'autoencoder

In [ ]:
def get_hidden(data):
    new_model = tf.keras.Model(inputs=autoencoder_model.input, outputs=[autoencoder_model.layers[1].output, autoencoder_model.output])
    hidden, output = new_model.predict(data)
    return hidden

def get_output(data):
    new_model = tf.keras.Model(inputs=autoencoder_model.input, outputs=[autoencoder_model.layers[1].output, autoencoder_model.output])
    hidden, output = new_model.predict(data)
    return output

Visualizzazione della relazione tra item e fattore

In [ ]:
hidden_layer = get_hidden(Dataset)
output_layer = get_output(Dataset)


plt.scatter(KSI[:,0],Dataset[:,0])
plt.scatter(-hidden_layer,output_layer[:,0])
plt.xlabel("factor scores")
plt.ylabel("observed scores")

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pc_array = pca.fit(Dataset).transform(Dataset) #ARRAY
proj_array = pca.inverse_transform(pc_array)

plt.scatter(KSI[:,0],Dataset[:,0])
plt.scatter(-pc_array,proj_array[:,0])
plt.xlabel("factor scores")
plt.ylabel("observed scores")

In [ ]:
mse_autoencoder = mean_squared_error(KSI[:,0],-hidden_layer[:,0])
print(mse_autoencoder)

In [ ]:
mse_pca = mean_squared_error(KSI[:,0],-pc_array[:,0])
print(mse_pca)

VISUALIZZAZIONE DELLA RICOSTRUZIONE DELLA RELAZIONE TRA ITEM E FATTORE DA PARTE DELL'AUTOENCODER

MSE DELLA RICOSTRUZIONE DELLA RELAZIONE TRA ITEM E FATTORE DA PARTE DELL'AUTOENCODER

---

